In [1]:
import pandas as pd
import numpy as np
import feedparser
from bs4 import BeautifulSoup
import requests
import PyPDF2 as pdf
import io
import os
import re
from gsheet import create
from gsheet import update

## Step1: Scrape the RSS Feed

In [130]:
# URL of the New Orleans RSS feed to scrape
rss_url = "https://www.mvn.usace.army.mil/DesktopModules/ArticleCS/RSS.ashx?ContentType=4&Site=417&isdashboardselected=0&max=2000"

# Parse the RSS feed using feedparser
rss_parsed = feedparser.parse(rss_url)

# Create a df to save everything into
rss_df = pd.DataFrame(rss_parsed.entries)

# Clean the df for New Orleans
rss_df = rss_df[["title", "link", "summary", "published"]].rename(
    columns = {"title": "web_title", "link": "web_link", "summary": "web_summary", "published": "published_date"}, 
    copy = False)

rss_df

,web_title,web_link,web_summary,published_date
0,MVN-2022-00799-CF,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED RV PARK ALONG PORT ALLEN LOCK<br />,"Mon, 21 Aug 2023 04:00:00 GMT"
1,MVN-2023-00140-ECS,https://www.mvn.usace.army.mil/Missions/Regula...,CRAWFORD DRAINAGE CANAL IMPROVEMENTS IN ST.CHA...,"Mon, 21 Aug 2023 04:00:00 GMT"
2,MVN-2022-00476-ECS,https://www.mvn.usace.army.mil/Missions/Regula...,MULTI-FAMILY RESIDENTIAL DEVELOPMENT IN ST.TAM...,"Mon, 21 Aug 2023 04:00:00 GMT"
3,MVN-2022-01256-WPP,https://www.mvn.usace.army.mil/Missions/Regula...,Proposed Residential Subdivision Development i...,"Mon, 21 Aug 2023 04:00:00 GMT"
4,MVN-1999-02360-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,TIME EXTENSION FOR THE ENGLISH TURN DEVELOPMEN...,"Mon, 14 Aug 2023 04:00:00 GMT"
...,...,...,...,...
495,MVN 2006-00106-CF,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED McHUGH ROAD IMPROVEMENT AND REALIGNME...,"Mon, 11 May 2020 04:00:00 GMT"
496,MVN-2018-00991-CG,https://www.mvn.usace.army.mil/Missions/Regula...,PARKING LOT AND LAYDOWN AREA FOR AN EXITING FA...,"Mon, 11 May 2020 04:00:00 GMT"
497,MVN 2006-01909 EMM,https://www.mvn.usace.army.mil/Missions/Regula...,"CLEAR, GRADE, EXCAVATE, AND FILL FOR A HOMESIT...","Thu, 07 May 2020 22:48:50 GMT"
498,MVN 2005-00618 EMM,https://www.mvn.usace.army.mil/Missions/Regula...,"CLEAR, GRADE, EXCAVATE, AND FILL FOR A SUBDIVI...","Thu, 07 May 2020 22:47:28 GMT"


## Step2: Scrape the webpage

### Define scraping functions

In [140]:
def extract_pdf_url(soup):
    try:
        pdf_end = soup.findAll('a', {"class": "link"})[2]['href']
        pdf_url = "https://www.mvn.usace.army.mil" + pdf_end
    except Exception as e:
        pdf_url = "ERROR: " + str(e)
    finally:
        return pdf_url


def extract_expire_date(soup):
    try:
        expire_date = re.search(r'(?<=:\s).+', soup.find_all("div", "expire")[0].get_text()).group()
    except Exception as e:
        expire_date  = "ERROR: " + str(e)
    finally:
        return expire_date


def extract_special_notice(soup):
    try:
        notice_type = soup.find_all("a", {"class":"link"})[2].get_text()
        if "Special" in notice_type:
            special_public_notice = "Yes"   
        else:
            special_public_notice = "No"
    except Exception as e:
        special_public_notice = "ERROR: " + str(e)
    finally:
        return special_public_notice

    
def extract_body(soup):
    try:
        body = soup.find_all("div", {"itemprop": "articleBody"})[0]
        if body.find("p") is None:
            body_text = body.get_text()
        else:
            body_text = body.get_text().replace(u'\xa0', u'').replace("\n", "")
    except:
        body_text = "Error"
    finally:
        return body_text

    
def extract_applicant(body_text):
    # Get all info in "NAME OF APPLICANT"
    try:
        applicant_contents = re.search(r'(?<=(APPLICANT|Applicant):)\s*.+(?=(LOCATION|Location))', body_text).group().strip()
        # Extract applicant and contractor
        if applicant_contents.find("c/o") != -1:
            try:
                applicant = re.search(r'.+?(?=\,* c/o)', applicant_contents).group().strip()
            except Exception as e:
                applicant = "ERROR: " + str(e)
            try:    
                contractor = re.search(r'(?<=c/o( |:)).+?(?=(,?\s?Post|,?\s?PO|,?\s?P\.O\.|,?\s*\d|,?\s?[Aa][tT]))', applicant_contents).group().strip()
            except Exception as e:
                contractor = "ERROR: " + str(e)
        else:
            contractor = None
            try:
                applicant = re.search(r'.+?(?=(,?\s?Post|,?\s?PO|,?\s?P\.O\.|,?\s?\d|,?\s?[Aa][tT]))', applicant_contents).group().strip()
            except Exception as e:
                applicant = "ERROR: " + str(e)   
    except Exception as e:
        applicant_contents = applicant = contractor = "Error: " + str(e)
    finally:
        return applicant_contents, applicant, contractor

def extract_location(body_text):
    try:
        location = re.search(r'(?<=(LOCATION OF WORK|Location of Work):)\s*.+(?=(CHARACTER OF WORK|Character of Work))', body_text).group().strip()
    except Exception as e:
        location = "ERROR: " + str(e)
    finally:
        return location
        
    
def extract_character_mitigation(body_text):
    if body_text.find(r'(Mitigation|MITIGATION)') != -1:
        try:
            work_character = re.search(r'(?<=(CHARACTER OF WORK|Character of Work):)\s*.+(?=(MITIGATION|Mitigation))', body_text).group().strip()
        except Exception as e:
            work_character = "ERROR: " + str(e)
        try:
            mitigation = re.search(r'(?<=(MITIGATION|Mitigation):)\s*.+', body_text).group().strip()
        except Exception as e:
            mitigation = "ERROR: " + str(e)
    else:
        mitigation = None                                     
        try:
            work_character = re.search(r'(?<=(CHARACTER OF WORK|Character of Work):)\s*.+', body_text).group().strip()
        except Exception as e:
            work_character = "ERROR: " + str(e)
    return work_character, mitigation        

    
def extract_hydrologic_unit_code(body_text):
    if body_text.find(r'Hydrologic Unit Code') != -1:
        try:
            hydrologic_unit_code = re.search(r'(?<=Hydrologic Unit Code(:|\s))[\s|\d]*', body_text).group().strip()
        except Exception as e:
            hydrologic_unit_code = "ERROR: " + str(e)
    else:
        hydrologic_unit_code = None
    return hydrologic_unit_code

def web_extraction(web_url):
    
    req = requests.get(web_url)
    content = req.text
    soup = BeautifulSoup(content, 'html.parser')

    # Get the pdf links
    pdf_url = extract_pdf_url(soup)

    # Check if the public notice is "Special Public Notice" or "Joint Public Notice"
    special_public_notice = extract_special_notice(soup)

    # Get expiration date
    expire_date = extract_expire_date(soup)

    # Extract webpage body
    body_text = extract_body(soup)

    if body_text != "Error":

        # Get applicant and contractor
        applicant = extract_applicant(body_text)

        # Get location
        location = extract_location(body_text)

        # Get character of work and mitigation if any
        character_mitigation = extract_character_mitigation(body_text)

        # Get location
        hydrologic_unit_code = extract_hydrologic_unit_code(body_text)
        #lon = re.findall(r'(?<=[-W])\s*\d{2}\.\d+', location)
        #lat = re.findall(r"(?<=[^-W][^-\d])\d{2}\.\d+", location)

        # Get details from character of work

    else:
        # Assign "Error" to all fields inside of website body.
        applicant = "ERROR", "ERROR", "ERROR"
        location =  hydrologic_unit_code = "ERROR"
        character_mitigation = "ERROR", "ERROR"
        
    return [pdf_url, special_public_notice, expire_date, applicant[0], applicant[1], applicant[2], \
 location, character_mitigation[0], character_mitigation[1], hydrologic_unit_code]

### Get the extracted results

In [175]:
web_df = pd.DataFrame([web_extraction(x) for x in rss_df["web_link"]],
                            columns = ['pdf_url',
                                       'special_public_notice',
                                       'expiration_date',  
                                       'applicant_full_info', 
                                       'applicant', 
                                       'contractor', 
                                       'location', 
                                       'work_character', 
                                       'mitigation', 
                                       'hydrologic_unit_code'])
# Merge with RSS feed table
rss_web_df = rss_df.join(web_df)

# Add a new column to track the number of errors for each webpage
rss_web_df["error"] = rss_web_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

rss_web_df

,web_title,web_link,web_summary,published_date,pdf_url,special_public_notice,expiration_date,applicant_full_info,applicant,contractor,location,work_character,mitigation,hydrologic_unit_code,error
0,MVN-2022-00799-CF,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED RV PARK ALONG PORT ALLEN LOCK<br />,"Mon, 21 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/11/2023,"Scott Thomas Development, LLC c/o: Trusted Com...","Scott Thomas Development, LLC","Trusted Compliance, LLC","Located in Brusly, Louisiana, within West Bato...",The applicant has requested Department of the ...,None,08070300,0
1,MVN-2023-00140-ECS,https://www.mvn.usace.army.mil/Missions/Regula...,CRAWFORD DRAINAGE CANAL IMPROVEMENTS IN ST.CHA...,"Mon, 21 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/11/2023,"St. Charles Parish Government, c/o: Civix, Ms....",St. Charles Parish Government,"Civix, Ms. Jane Dufour","Located in Section 20, Township 15 South, Rang...",The proposed project is to widen the existing ...,None,080903010302,0
2,MVN-2022-00476-ECS,https://www.mvn.usace.army.mil/Missions/Regula...,MULTI-FAMILY RESIDENTIAL DEVELOPMENT IN ST.TAM...,"Mon, 21 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/11/2023,"STOA Group, c/o: Hydrik Wetlands Consultants, ...",STOA Group,"Hydrik Wetlands Consultants, Mr. Michael Henry","Located northerly of McKinney Road, Slidell, i...","The proposed project includes the clearing, gr...",None,080902010307,0
3,MVN-2022-01256-WPP,https://www.mvn.usace.army.mil/Missions/Regula...,Proposed Residential Subdivision Development i...,"Mon, 21 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/11/2023,"Morganfield South, LLC, c/o: Rhett Francois, 1...","Morganfield South, LLC",Rhett Francois,"Located in Calcasieu Parish, near the communit...","Clear, grade, excavate and fill (approximately...",None,08080206,0
4,MVN-1999-02360-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,TIME EXTENSION FOR THE ENGLISH TURN DEVELOPMEN...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/4/2023,"English Turn Limited Partnership, Attention: J...",English Turn Limited Partnership,None,"At approximately Latitude: 29.90826, Longitude...",Continual operations to install and maintain f...,None,08090301,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MVN 2006-00106-CF,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED McHUGH ROAD IMPROVEMENT AND REALIGNME...,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"CITY OF BATON ROUGE, PARISH OF EAST BATON ROUG...","CITY OF BATON ROUGE, PARISH OF EAST BATON ROUGE","CSRS, Inc.","In South Canal, Latitude 30.605043, Longitude ...",The applicant has requested a Department of th...,None,None,0
496,MVN-2018-00991-CG,https://www.mvn.usace.army.mil/Missions/Regula...,PARKING LOT AND LAYDOWN AREA FOR AN EXITING FA...,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"PCS Nitrogen Fertilizer, LP., c/o Olivia Barry...","PCS Nitrogen Fertilizer, LP.",Olivia Barry of CK Associ,At the existing PCS Nitrogen facility in Secti...,The applicant has requested Department of the ...,None,None,0
497,MVN 2006-01909 EMM,https://www.mvn.usace.army.mil/Missions/Regula...,"CLEAR, GRADE, EXCAVATE, AND FILL FOR A HOMESIT...","Thu, 07 May 2020 22:48:50 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,6/8/2020,"Chad Bordelon, 1645 North Causeway Boulevard, ...",Chad Bordelon,None,"Section 27, Township 11 South, Range 7 East, i...","Clear, grade, excavate, and fill to construct ...",None,None,0
498,MVN 2005-00618 EMM,https://www.mvn.usace.army.mil/Missions/Regula...,"CLEAR, GRADE, EXCAVATE, AND FILL FOR A SUBDIVI...","Thu, 07 May 2020 22:47:28 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,6/1/2020,"Jean Lafitte Landing, LLC, c/o Hydrik Wetlan

## Step3: Scrap PDFs

In [ ]:
# Read in PDF as texts
pdf_test_url = "https://www.mvn.usace.army.mil/Portals/56/docs/regulatory/publicnotices/2023_00295_PNall_04172023.pdf?ver=cV8AZIWwrr8nk9bcqhvo3Q%3d%3d"

# Download the PDF content as a bytes object
response = requests.get(pdf_test_url)
pdf_bytes = response.content

# Create a PyPDF2 PdfFileReader object from the PDF content
pdf_reader = pdf.PdfFileReader(io.BytesIO(pdf_bytes))

# Extract text from all pages in the PDF file
text = ""
for page_num in range(pdf_reader.getNumPages()):
    page = pdf_reader.getPage(page_num)
    text += page.extractText()

# Print the extracted text
print(text)

## Report the results to Google Sheet

In [181]:
# Make the first row in the dataset to have same name as column (in order for the google sheets to have appropriate column names)
new_header = pd.DataFrame({'web_title': ['web_title'],
                           'web_link': ['web_link'],
                           'web_summary': ['web_summary'],
                           'published_date': ['published_date'],
                           'pdf_url': ['pdf_url'],
                           'special_public_notice': ['special_public_notice'],
                           'expiration_date': ['expiration_date'],
                           'applicant_full_info': ['applicant_full_info'],
                           'applicant': ['applicant'],
                           'contractor': ['contractor'],
                           'location': ['location'],
                           'work_character': ['work_character'],
                           'mitigation': ['mitigation'], 
                           'hydrologic_unit_code': ['hydrologic_unit_code'],
                           'error': ['error']})

# Concatenate the new row with the existing DataFrame
rss_web_gs = pd.concat([new_header, rss_web_df], ignore_index = True)

# Summary of webpage links having errors
error_summary = pd.concat([new_header, rss_web_df.loc[(rss_web_df["error"] > 0)]], ignore_index = True)

In [182]:
# Create a Google sheet to record the links with errors (Already created)
# if __name__ == '__main__':
#     # Pass: title
#     create("usace-test")

# I convert the lon/lat to string just for the storga in google sheet
# The format of lon/lat is still needed to be decided

# Update the scraped RSS feed and webpage info to the sheet
if __name__ == '__main__':
    # Pass: spreadsheet_id, range_name value_input_option and _values)
    update("1VYcmSCuvBMiRRpUusuulIcS4kjkXrTuFZLfKOV_IxkE",
            "new_orleans", 
           "USER_ENTERED",
           rss_web_gs.values.tolist())

# Update the error messages to the sheet
if __name__ == '__main__':
    update("1VYcmSCuvBMiRRpUusuulIcS4kjkXrTuFZLfKOV_IxkE",
            "error", 
           "USER_ENTERED",
           error_summary.values.tolist())

6667 cells updated.
873 cells updated.


## Achrived

In [3]:
# Loop
error_collect = pd.DataFrame(columns = ["error_title", "error_link", "error_message"])

# %%time
for row in rss_feed.index:
    try:
        main_url = rss_feed.iloc[row, 1]
        req = requests.get(main_url)
        content = req.text # json structure
        soup = BeautifulSoup(content)
        
        # Get the pdf links
        
        def pdf_url():
            try:
                pdf_end = soup.findAll('a', {"class": "link"})[2]['href']
                pdf_url = "https://www.mvn.usace.army.mil" + pdf_end
            except Exception as pdf_error:
                print("ERROR when getting pdf links", pdf_error)
                
        # Check if the public notice is "Special Public Notice" or "Joint Public Notice"

        public_notice = soup.find_all("a", {"class":"link"})[2].get_text()

        if public_notice != "Public Notice and Drawings":
            expire_date = None
            applicant = None
            contractor = None
            location = None
            work_character = None
            mitigation = None
        else: 

            # Get expiration date

            expire_date = re.search(r'(?<=:\s).+', soup.find_all("div", "expire")[0].get_text()).group()

            # Extract webpage body: two types of structures

            body = soup.find_all("div", {"itemprop":"articleBody"})[0]
            if body.find("p") is None:
                body_text = body.get_text()  
            else:
                body_text = body.get_text().replace(u'\xa0', u'').replace("\n", "")

            applicant_contents = re.search(r'(?<=APPLICANT:)\s*.+(?=LOCATION)', body_text).group().strip()
            location = re.search(r'(?<=LOCATION OF WORK:)\s*.+(?=CHARACTER OF WORK)', body_text).group().strip()

            if "MITIGATION" in body_text:
                work_character = re.search(r'(?<=CHARACTER OF WORK:)\s*.+(?=MITIGATION)', body_text).group().strip()
                mitigation = re.search(r'(?<=MITIGATION:)\s*.+', body_text).group().strip()
            else:
                work_character = re.search(r'(?<=CHARACTER OF WORK:)\s*.+', body_text).group().strip()
                mitigation = None

            # Get applicant name and permitting process contractor

            if applicant_contents.find("c/o") != -1:
                applicant = re.search(r'.+?(?=\,* c/o)', applicant_contents).group().strip()
                # if any(w in applicant_contents for w in ["Attention", "Attn"]):
                contractor = re.search(r'(?<=c/o( |:)).+?(?=(, Post|, PO|, P\.O\.|, \d|,? [Aa][tT]))', applicant_contents).group().strip()
                # else:
                #     contractor = re.search(r'(?<=c/o( |:))\D+(?=(, |, P.O.))', applicant_contents).group()
            else:
                contractor = None
                # if any(w in applicant_contents for w in ["Attention", "Attn"]):
                applicant = re.search(r'.+?(?=(, Post|, PO|, P\.O\.|, \d|,? [Aa][tT]))', applicant_contents).group().strip()
                # else:
                #     applicant = re.search(r'\D+(?=, )', applicant_contents).group()

            # Get location
            
            
            #lon = re.findall(r'(?<=[-W])\s*\d{2}\.\d+', location)
            #lat = re.findall(r"(?<=[^-W][^-\d])\d{2}\.\d+", location)
            
        rss_feed.loc[row, empty_columns] = np.array([pdf_url, expire_date, applicant_contents, applicant, contractor, location, lon, lat, work_character, mitigation], dtype = "object")
    
    except Exception as error_message:
        error_row = [rss_feed.iloc[row, 0], rss_feed.iloc[row, 1], repr(error_message)]
        error_collect.loc[len(error_collect.index)] = error_row
        # print("ERROR: ", error_link, error)

rss_feed

,title,link,desc,date,pdf,expiration_date,applicant_full_info,applicant,contractor,location,lon,lat,work_character,mitigation
0,MVN-1999-02360-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,TIME EXTENSION FOR THE ENGLISH TURN DEVELOPMEN...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,"English Turn Limited Partnership, Attention: J...",English Turn Limited Partnership,None,"At approximately Latitude: 29.90826, Longitude...",[89.952013],[29.90826],Continual operations to install and maintain f...,None
1,MVN-2020-00234-WPP,https://www.mvn.usace.army.mil/Missions/Regula...,MODIFICATION FOR MAINTENANCE DREDGING OF NAVIG...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,"Port of Iberia, c/o: GIS Engineering, LLC, 450...",Port of Iberia,"GIS Engineering, LLC","Located in Iberia Parish, Section 34, Township...","[91.8412, 92.3127306]","[29.958239, 29.5109862]",A Department of Army permit was issued on Apri...,None
2,MVN-2021-00560-WQQ,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED CP2 LIQUEFIED NATURAL GAS FACILITY AN...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,"Venture Global CP2 LNG, LLC and Venture Global...","Venture Global CP2 LNG, LLC and Venture Global...",Environmental Resources Management,The proposed facility and pipeline are located...,[],"[85.4, 58.1]",To construct and operate (1) natural gas lique...,None
3,MVN-2023-00058-WII,https://www.mvn.usace.army.mil/Missions/Regula...,TEST WELLS FOR CARBON DIOXIDE STORAGE IN VERMI...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,ExxonMobil Low Carbon Solutions Onshore Storag...,ExxonMobil Low Carbon Solutions Onshore Storag...,Duplantis Design Group,Test Well #1 @ Lat. 29-46-57.31 N / Long. -92-...,"[57.31, 56.39, 44.82, 37.40]",[],Proposed geological investigations via install...,None
4,MVN-2008-01186-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,AFTER-THE-FACT WEIGH STATION AND TRUCK LOADING...,"Mon, 07 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,8/28/2023,"KV Enterprises, LLC, c/o Professional Engineer...","KV Enterprises, LLC",Professional Engineering & Environmental Consu...,"At approximately Latitude: 30.00584, Longitude...",[90.51464],[30.00584],"Clear, grade, excavate, and fill for a weigh s...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MVN-2020-00422-WS,https://www.mvn.usace.army.mil/Missions/Regula...,"BULKHEAD REPLACEMENT IN TERREBONNE PARISH, LOU...","Mon, 04 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,5/24/2020,"Savage Inland Marine, LLC, c/o: Mr. David Boud...","Savage Inland Marine, LLC",Mr. David Boudreaux,"Located in Terrebonne Parish, Section 14, Town...",[91.018125],[29.665289],The applicant proposes to install approximatel...,None
496,MVN-2019-00377-WI,https://www.mvn.usace.army.mil/Missions/Regula...,CONSTRUCT ONE BRINE PIPELINE AND ONE ETHYLENE ...,"Mon, 04 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,5/23/2020,"Boardwalk Louisiana Midstream, LLC, c/o Provid...","Boardwalk Louisiana Midstream, LLC",Providence Engineering and Environmental Group...,"In Iberville Parish, North 30.316461, West -91...",[91.304178],[30.316461],"Clear, grade, trench and temporarily backfill ...",None
497,MVN 2006-01716-CM,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED CONSTRUCTION OF A RESIDNETIAL SUBDIVI...,"Mon, 04 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,5/24/2020,"J. Breaux Enterprises, c/o Hydrik Wetlands Con...",J. Breaux Enterprises,Hydrik Wetlands Consultants,"Between West Colyell Creek and Beaver Branch, ...",[90.881303],[30.57454],The applicant has requested Department of the ...,None
498,MVN-2020-00393-CE,https://www.mvn.usace.army.mil/Missions/Regula...,MODIFICATION OF AN EXISTING BARGE FLEETING OPE...,"Mon, 